In [2]:
%load_ext autoreload
%autoreload 2
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from pylexique import Lexique383
import os
from pathlib import Path
import data
import pandas as pd
import xgboost as xgb

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
current_path = os.path.abspath('')
BASE_PATH = Path(current_path)
def get_absolute_file_path(path):
    return os.path.join(BASE_PATH, path)

In [4]:
train_data_file_name = get_absolute_file_path('data/train.csv')
train_data = pd.read_csv(train_data_file_name)

In [11]:
train_data = data.preprocess(train_data, create_keywords=True, clean_text=True, count_abreviations=False)

In [7]:
LEXIQUE = Lexique383()
lexique = set(LEXIQUE.lexique.keys())

In [9]:
train_data["abrev"] = train_data["text"].apply(lambda s: data.count_abreviations_func(s, lexique))

In [17]:
# select useful columns
# train_data_filtered = train_data.drop(["text", "urls", "mentions", "hashtags", "timestamp", "TweetID"], axis=1)
train_data_filtered = train_data.loc[:, ["retweets_count","favorites_count","followers_count","statuses_count","friends_count",
                                 "hashtags_count","hour","verified","url_count","text_len","rt","Macron","Zemmour","Melenchon"]]

In [18]:
X_train, X_eval, y_train, y_eval = train_test_split(train_data_filtered.drop("retweets_count", axis=1),
                                                    train_data_filtered["retweets_count"],
                                                    random_state=42, test_size=0.2)

# Standardize the data
normal_columns = train_data_filtered.drop(["hour", "verified", "Macron", "Zemmour", "Melenchon", "url_count", "rt", "retweets_count"], axis=1).columns
mu, sigma = X_train[normal_columns].mean(axis=0), X_train[normal_columns].std(axis=0)
X_train.loc[:, normal_columns] = (X_train[normal_columns] - mu) / sigma
X_eval.loc[:, normal_columns] = (X_eval[normal_columns] - mu) / sigma

In [19]:
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.300000012, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=6, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=42, ...)

In [20]:
y_pred = xgb_model.predict(X_eval)
mae = mean_absolute_error(y_eval, y_pred)
print(mae)

7.1741439594374885
